In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Jun  4 20:36:53 2021

@author: chendanlei

original template: SC_combined_Wager_atlas_edited.nii.gz which is my drawing based on Tor's atlas
mask by 
1. probabilistic grey matter mask thresholded at >=0.25
2. probabilistic white matter mask thresholded at >=0.25
3. probabilistic CSF mask thresholded at <=0.5
4. binary aqueduct mask at ==1
5. probabilistic pag mask thresholded at 0.2
6. sigmasquared map thesholded at <85 percentile
7. cluster >10 voxels
"""

import nibabel as nib
import numpy as np
import glob
from nilearn.glm import threshold_stats_img
from nilearn.image import resample_img
import pandas as pd 
import os


/Users/chendanlei/.local/lib/python3.7/site-packages/nilearn/glm/__init__.py:56: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  'It may change in any future release of Nilearn.', FutureWarning)


In [94]:
save_dir = '/Volumes/GoogleDrive/My Drive/U01/working_memory/roi/subject_SC_mask/subject_SC_mask/'
wager_SC_mask = '/Volumes/GoogleDrive/My Drive/U01/working_memory/roi/subject_SC_mask/SC_combined_Wager_atlas_edited.nii.gz'
subj_list_all = pd.read_csv('/Volumes/GoogleDrive/My Drive/U01/AffPainTask_connectivity/analysis/cleanup/EmoPain_good_run_all.csv', sep='\t', header=None)[0]
# subj_list_task = list([i for i in subj_list_all if '3_run' in i])
# subj_list = list(set([i.split('_task')[0] for i in subj_list_task]))
# run_list = run_list[1:2]
# subj_list_task.sort()
subj_list = ["sub-022", "sub-043", "sub-050", "sub-057", "sub-063", "sub-069", "sub-078", "sub-085", "sub-092", "sub-099", "sub-106", "sub-124", "sub-134",  "sub-012", "sub-023", "sub-032", "sub-044", "sub-052", "sub-058", "sub-064", "sub-070", "sub-080", "sub-086", "sub-093", "sub-100", "sub-110", "sub-117", "sub-135", "sub-013", "sub-024", "sub-034", "sub-045", "sub-053", "sub-059", "sub-065", "sub-071", "sub-081", "sub-087", "sub-094", "sub-101", "sub-111", "sub-118", "sub-127", "sub-015", "sub-025", "sub-037", "sub-047", "sub-054", "sub-060", "sub-066", "sub-072", "sub-082", "sub-088", "sub-095", "sub-102", "sub-112", "sub-119", "sub-128", "sub-137", "sub-019", "sub-026", "sub-039", "sub-048", "sub-055", "sub-061", "sub-067", "sub-073", "sub-083", "sub-090", "sub-104", "sub-120", "sub-131", "sub-138", "sub-021", "sub-030", "sub-042", "sub-049", "sub-056", "sub-062", "sub-074", "sub-084", "sub-091", "sub-098", "sub-105", "sub-114", "sub-122", "sub-133", "sub-139"]
# subj_list=['sub-047b']
subj_list.sort()

# #make symmetrical SC template
# wager_SC_mask = '/Users/chendanlei/Google Drive/U01/AffPainTask_connectivity/roi/subject_SC_mask/SC_combined_Wager_atlas_edited.nii.gz'
# roi = nib.load(wager_SC_mask).get_fdata()
# # roi[int(roi.shape[0]/2):,:,:]==1
# for n,x in enumerate(range(int(roi.shape[0]/2),int(roi.shape[0]))):
#     roi[98-(n),:,:]=roi[98+(n),:,:]
# test_img = nib.Nifti1Image(roi, nib.load(wager_SC_mask).affine, nib.load(wager_SC_mask).header)
# nib.save(test_img, '/Users/chendanlei/Google Drive/U01/AffPainTask_connectivity/roi/subject_SC_mask/test.nii.gz')

mask_data = nib.load('/Users/chendanlei/Desktop/U01/level1_files/emoAvd_CSUSnegneu/SC_warp/sub-014_run-01/wdata_sub-014_run-01_cope1.nii.gz')


In [90]:
subj_list=['sub-125b']
for subj in subj_list:

    print(subj)
    
    #determine if subject is pain or emo
    #download pag mask, aqueduct mask, sigmasquared

    # pag_mask = '/Users/chendanlei/Desktop/sub-011_run-01_pag_mask.nii'
    # aqueduct_mask = '/Users/chendanlei/Desktop/sub-011_run-01_aqueduct_template.nii.gz'
    os.system('rsync -r --ignore-existing dz609@door.nmr.mgh.harvard.edu:/autofs/cluster/iaslab/FSMAP/FSMAP_data/BIDS_modeled/extract_PAG_wm/templates/'+subj+'*.nii.gz /Volumes/GoogleDrive/My\ Drive/U01/working_memory/roi/subject_SC_mask/PAG')
    os.system('rsync -r --ignore-existing dz609@door.nmr.mgh.harvard.edu:/autofs/cluster/iaslab/FSMAP/FSMAP_data/BIDS_modeled/extract_PAG_wm_step2/pag_mask/'+subj+'*.nii /Volumes/GoogleDrive/My\ Drive/U01/working_memory/roi/subject_SC_mask/PAG')
    os.system('rsync --ignore-existing dz609@door.nmr.mgh.harvard.edu:/autofs/cluster/iaslab/FSMAP/FSMAP_data/BIDS_modeled/BIDS_modeled/wm_nback_model/nosmooth/'+subj+'/model/*/_modelestimate0/sigmasquareds.nii.gz /Volumes/GoogleDrive/My\ Drive/U01/working_memory/roi/subject_SC_mask/sigmasquared/')
    os.system('mv /Volumes/GoogleDrive/My\ Drive/U01/working_memory/roi/subject_SC_mask/sigmasquared/sigmasquareds.nii.gz /Volumes/GoogleDrive/My\ Drive/U01/working_memory/roi/subject_SC_mask/sigmasquared/'+subj+'_sigmasquareds.nii.gz')
#     aqueduct_mask = glob.glob('/Volumes/GoogleDrive/My Drive/U01/working_memory/roi/subject_SC_mask/PAG/'+subj+'*.nii.gz')[0]
#     pag_mask = glob.glob('/Volumes/GoogleDrive/My Drive/U01/working_memory/roi/subject_SC_mask/PAG/'+subj+'*.nii')[0]
#     sigmasquared_mask = glob.glob('/Volumes/GoogleDrive/My Drive/U01/working_memory/roi/subject_SC_mask/sigmasquared/'+subj+'*.nii.gz')[0]

    #download gm, wm, csf maps
    os.system('rsync -r --ignore-existing dz609@door.nmr.mgh.harvard.edu:/autofs/cluster/iaslab/FSMAP/FSMAP_data/BIDS_fmriprep/fmriprep/ses-01/'+subj+'/anat/*_T1w_space-MNI152NLin2009cAsym_class-GM_probtissue.nii.gz /Volumes/GoogleDrive/My\ Drive/U01/working_memory/roi/subject_SC_mask/T1')
    os.system('rsync -r --ignore-existing dz609@door.nmr.mgh.harvard.edu:/autofs/cluster/iaslab/FSMAP/FSMAP_data/BIDS_fmriprep/fmriprep/ses-01/'+subj+'/anat/*_T1w_space-MNI152NLin2009cAsym_class-WM_probtissue.nii.gz /Volumes/GoogleDrive/My\ Drive/U01/working_memory/roi/subject_SC_mask/T1')
    os.system('rsync -r --ignore-existing dz609@door.nmr.mgh.harvard.edu:/autofs/cluster/iaslab/FSMAP/FSMAP_data/BIDS_fmriprep/fmriprep/ses-01/'+subj+'/anat/*_T1w_space-MNI152NLin2009cAsym_class-CSF_probtissue.nii.gz /Volumes/GoogleDrive/My\ Drive/U01/working_memory/roi/subject_SC_mask/T1')
# #     gm_map = '/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/T1/'+subj+'_T1w_space-MNI152NLin2009cAsym_class-GM_probtissue.nii.gz'
# #     wm_map = '/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/T1/'+subj+'_T1w_space-MNI152NLin2009cAsym_class-WM_probtissue.nii.gz'
# #     csf_map = '/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/T1/'+subj+'_T1w_space-MNI152NLin2009cAsym_class-CSF_probtissue.nii.gz'
    



sub-125b


In [95]:
for n,subj in enumerate(subj_list[85:]):
    print(n)
    print(subj)
    
    #determine if subject is pain or emo
    #download pag mask, aqueduct mask, sigmasquared

    # pag_mask = '/Users/chendanlei/Desktop/sub-011_run-01_pag_mask.nii'
    # aqueduct_mask = '/Users/chendanlei/Desktop/sub-011_run-01_aqueduct_template.nii.gz'
#     os.system('rsync -r --ignore-existing dz609@door.nmr.mgh.harvard.edu:/autofs/cluster/iaslab/FSMAP/FSMAP_data/BIDS_modeled/extract_PAG_wm/templates/'+subj+'*.nii.gz /Volumes/GoogleDrive/My\ Drive/U01/working_memory/roi/subject_SC_mask/PAG')
#     os.system('rsync -r --ignore-existing dz609@door.nmr.mgh.harvard.edu:/autofs/cluster/iaslab/FSMAP/FSMAP_data/BIDS_modeled/extract_PAG_wm_step2/pag_mask/'+subj+'*.nii /Volumes/GoogleDrive/My\ Drive/U01/working_memory/roi/subject_SC_mask/PAG')
#     os.system('rsync --ignore-existing dz609@door.nmr.mgh.harvard.edu:/autofs/cluster/iaslab/FSMAP/FSMAP_data/BIDS_modeled/BIDS_modeled/wm_nback_model/nosmooth/'+subj+'/model/*/_modelestimate0/sigmasquareds.nii.gz /Volumes/GoogleDrive/My\ Drive/U01/working_memory/roi/subject_SC_mask/sigmasquared/')
#     os.system('mv /Volumes/GoogleDrive/My Drive/U01/working_memory/roi/subject_SC_mask/sigmasquared/sigmasquareds.nii.gz /Volumes/GoogleDrive/My\ Drive/U01/working_memory/roi/subject_SC_mask/sigmasquared/'+subj+'_sigmasquareds.nii.gz')
    aqueduct_mask = glob.glob('/Volumes/GoogleDrive/My Drive/U01/working_memory/roi/subject_SC_mask/PAG/'+subj+'*.nii.gz')[0]
    pag_mask = glob.glob('/Volumes/GoogleDrive/My Drive/U01/working_memory/roi/subject_SC_mask/PAG/'+subj+'*.nii')[0]
    sigmasquared_mask = glob.glob('/Volumes/GoogleDrive/My Drive/U01/working_memory/roi/subject_SC_mask/sigmasquared/'+subj+'*.nii.gz')[0]

    #download gm, wm, csf maps
#     os.system('rsync -r --ignore-existing dz609@door.nmr.mgh.harvard.edu:/autofs/cluster/iaslab/FSMAP/FSMAP_data/BIDS_fmriprep/fmriprep/ses-01/'+subj+'/anat/*_T1w_space-MNI152NLin2009cAsym_class-GM_probtissue.nii.gz /Volumes/GoogleDrive/My\ Drive/U01/working_memory/roi/subject_SC_mask/T1')
#     os.system('rsync -r --ignore-existing dz609@door.nmr.mgh.harvard.edu:/autofs/cluster/iaslab/FSMAP/FSMAP_data/BIDS_fmriprep/fmriprep/ses-01/'+subj+'/anat/*_T1w_space-MNI152NLin2009cAsym_class-WM_probtissue.nii.gz /Volumes/GoogleDrive/My\ Drive/U01/working_memory/roi/subject_SC_mask/T1')
#     os.system('rsync -r --ignore-existing dz609@door.nmr.mgh.harvard.edu:/autofs/cluster/iaslab/FSMAP/FSMAP_data/BIDS_fmriprep/fmriprep/ses-01/'+subj+'/anat/*_T1w_space-MNI152NLin2009cAsym_class-CSF_probtissue.nii.gz /Volumes/GoogleDrive/My\ Drive/U01/working_memory/roi/subject_SC_mask/T1')
    gm_map = '/Volumes/GoogleDrive/My Drive/U01/working_memory/roi/subject_SC_mask/T1/'+subj+'_T1w_space-MNI152NLin2009cAsym_class-GM_probtissue.nii.gz'
    wm_map = '/Volumes/GoogleDrive/My Drive/U01/working_memory/roi/subject_SC_mask/T1/'+subj+'_T1w_space-MNI152NLin2009cAsym_class-WM_probtissue.nii.gz'
    csf_map = '/Volumes/GoogleDrive/My Drive/U01/working_memory/roi/subject_SC_mask/T1/'+subj+'_T1w_space-MNI152NLin2009cAsym_class-CSF_probtissue.nii.gz'
    
    print('masking GM, WM, and CSF maps')
    gm_map_img = nib.load(gm_map)
    gm_wm_csf_mask_data = nib.load(gm_map).get_fdata()
    gm_wm_csf_mask_data[gm_wm_csf_mask_data>=0.25] = 1
    gm_wm_csf_mask_data[nib.load(wm_map).get_fdata()>=0.25] = 1
    gm_wm_csf_mask_data[nib.load(csf_map).get_fdata()>=0.5] = 0
    gm_wm_csf_mask_data[gm_wm_csf_mask_data!=1] = 0
    
    #resample SC, PAG, aqueduct, and sigmasquared to T1
    print('resampling everything to T1')
    SC_data = resample_img(nib.load(wager_SC_mask),
                 target_affine=gm_map_img.affine,
                 target_shape=gm_map_img.shape[0:3],
                 interpolation='nearest').get_fdata()
    pag_data = resample_img(nib.load(pag_mask),
                 target_affine=gm_map_img.affine,
                 target_shape=gm_map_img.shape[0:3],
                 interpolation='nearest').get_fdata()
    aqueduct_data = resample_img(nib.load(aqueduct_mask),
                 target_affine=gm_map_img.affine,
                 target_shape=gm_map_img.shape[0:3],
                 interpolation='nearest').get_fdata()
    sigmasquared_data = resample_img(nib.load(sigmasquared_mask),
                 target_affine=gm_map_img.affine,
                 target_shape=gm_map_img.shape[0:3],
                 interpolation='nearest').get_fdata()
    
    print('thresholding SC')
    #threshold SC based on T1
    SC_data[gm_wm_csf_mask_data==0] = 0
    #threshold SC based on pag
    SC_data[pag_data>0.2] = 0
    #threshold SC based on aqueduct
    SC_data[aqueduct_data!=0] = 0
    #threshold SC based on sigmasquared
    thresh = np.percentile(sigmasquared_data[sigmasquared_data>0], 85)
    SC_data[sigmasquared_data>thresh] = 0
    
    SC_img = nib.Nifti1Image(SC_data, gm_map_img.affine, gm_map_img.header)
    
    #get rid of left out small cluster
    print('clustering')
    SC_img_clust,t = threshold_stats_img(SC_img, cluster_threshold=10, threshold=0, height_control=None)
    
    #resample to the same size and affine as functional file
    SC_img_clust_resampled = resample_img(SC_img_clust,
        target_affine=mask_data.affine,
        target_shape=mask_data.shape[0:3],
        interpolation='nearest')
    
    print('saving')
    #save final subject SC roi
    nib.save(SC_img_clust_resampled, save_dir+wager_SC_mask.split('/')[-1].split('.nii')[0]+'_'+subj+'_'+'wm_masked_resampled.nii.gz')


0
sub-137
masking GM, WM, and CSF maps
resampling everything to T1
thresholding SC
clustering
saving
1
sub-138
masking GM, WM, and CSF maps
resampling everything to T1
thresholding SC
clustering
saving
2
sub-139
masking GM, WM, and CSF maps
resampling everything to T1
thresholding SC
clustering
saving


In [98]:
len(glob.glob('/Volumes/GoogleDrive/My Drive/U01/working_memory/roi/subject_SC_mask/subject_SC_mask/*'))

88

In [ ]:
rsync -r /Volumes/GoogleDrive/My\ Drive/U01/working_memory/roi/subject_SC_mask/subject_SC_mask/* dz609@door.nmr.mgh.harvard.edu:/autofs/cluster/iaslab/FSMAP2/FSMAP_data/BIDS_modeled/subject_SC_mask/wm
    

In [118]:
#check if two sides are in similar sizes
emoPain = 'emo'
data_dir = '/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/'+emoPain+'Avd/SC_mask/'
all_masks = glob.glob(data_dir+'*')
all_masks.sort()
for n,m in enumerate(all_masks):
    print(m)
    roi = nib.load(m).get_fdata()
    print(np.sum(roi[int(roi.shape[0]/2):,:,:]==1)-np.sum(roi[0:int(roi.shape[0]/2),:,:]==1))
    print()
    

/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_atlas_edited_sub-014_run-01_emo_masked.nii.gz
-38

/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_atlas_edited_sub-014_run-02_emo_masked.nii.gz
-32

/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_atlas_edited_sub-014_run-03_emo_masked.nii.gz
-31

/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_atlas_edited_sub-014_run-04_emo_masked.nii.gz
-32

/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_atlas_edited_sub-014_run-05_emo_masked.nii.gz
-27

/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_atlas_edited_sub-016_run-01_emo_masked.nii.gz
-31

/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_a

-17

/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_atlas_edited_sub-060_run-01_emo_masked.nii.gz
-16

/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_atlas_edited_sub-060_run-02_emo_masked.nii.gz
0

/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_atlas_edited_sub-060_run-03_emo_masked.nii.gz
-25

/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_atlas_edited_sub-060_run-04_emo_masked.nii.gz
-20

/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_atlas_edited_sub-060_run-05_emo_masked.nii.gz
-14

/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_atlas_edited_sub-061_run-01_emo_masked.nii.gz
-12

/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wage

-32

/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_atlas_edited_sub-099_run-01_emo_masked.nii.gz
-31

/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_atlas_edited_sub-099_run-02_emo_masked.nii.gz
-23

/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_atlas_edited_sub-099_run-03_emo_masked.nii.gz
-26

/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_atlas_edited_sub-099_run-04_emo_masked.nii.gz
-31

/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_atlas_edited_sub-099_run-05_emo_masked.nii.gz
-18

/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_atlas_edited_sub-100_run-01_emo_masked.nii.gz
-25

/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wa

28



In [43]:
#reslice all the ROIs based on cope template
emoPain = 'emo'
data_dir = '/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/'+emoPain+'Avd/SC_mask/'
mask_dir = '/Users/chendanlei/Desktop/U01/ROI/cope1.nii.gz'

all_files = glob.glob(data_dir+'*sub*')
all_files.sort()
for file_dir in all_files[0:5]:
    #resample mask_img to cope_img
    print(file_dir)
    print('resample mask')
    masked_file = resample_img(nib.load(file_dir),
        target_affine=nib.load(mask_dir).affine,
        target_shape=nib.load(mask_dir).shape[0:3],
        interpolation='linear')
    print('save masked image')
    new_name = file_dir.split(".nii")[-2] + '_resampled.nii.gz'
    nib.save(masked_file, new_name)


/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_atlas_edited_sub-014_run-01_emo_masked.nii.gz
resample mask


/Users/chendanlei/.local/lib/python3.7/site-packages/nilearn/image/resampling.py:273: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn("Resampling binary images with continuous or "


save masked image
/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_atlas_edited_sub-014_run-02_emo_masked.nii.gz
resample mask


/Users/chendanlei/.local/lib/python3.7/site-packages/nilearn/image/resampling.py:273: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn("Resampling binary images with continuous or "


save masked image
/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_atlas_edited_sub-014_run-03_emo_masked.nii.gz
resample mask


...

save masked image
/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_atlas_edited_sub-014_run-04_emo_masked.nii.gz
resample mask


/Users/chendanlei/.local/lib/python3.7/site-packages/nilearn/image/resampling.py:273: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn("Resampling binary images with continuous or "


save masked image
/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_atlas_edited_sub-014_run-05_emo_masked.nii.gz
resample mask


/Users/chendanlei/.local/lib/python3.7/site-packages/nilearn/image/resampling.py:273: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn("Resampling binary images with continuous or "


save masked image


In [38]:
file_dir

'/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/emoAvd/SC_mask/SC_combined_Wager_atlas_edited_sub-014_run-01_emo_masked_resampled.nii.gz'

In [41]:
nib.load(file_dir).affine

array([[   1.102     ,    0.        ,    0.        ,  -96.        ],
       [   0.        ,    1.102     ,    0.        , -132.        ],
       [   0.        ,    0.        ,    1.10000002,  -78.        ],
       [   0.        ,    0.        ,    0.        ,    1.        ]])

In [ ]:
#group average
emoPain = 'pain'
data_dir = '/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/'+emoPain+'Avd/SC_mask/'
save_dir = '/Users/chendanlei/Desktop/U01/ROI/subject_SC_mask/subject_SC_mask/'+emoPain+'Avd/'
all_masks = glob.glob(data_dir+'*')
for n,m in enumerate(all_masks):
    if n == 0:
        sum_mask = nib.load(m).get_fdata().copy()
    else:
        sum_mask = sum_mask + nib.load(m).get_fdata()
# sum_img = nib.Nifti1Image(sum_mask, nib.load(m).affine, nib.load(m).header)
# nib.save(sum_img, save_dir+wager_SC_mask.split('/')[-1].split('.nii')[0]+'_'+task+'_SUM_masked.nii.gz')

prob_mask = sum_mask/np.max(sum_mask)
prob_img = nib.Nifti1Image(prob_mask, nib.load(m).affine, nib.load(m).header)
nib.save(prob_img, m.split('_sub')[0]+'_'+emoPain+'_PROB_masked.nii.gz')

prob_mask[prob_mask<0.25] = 0
prob_mask[prob_mask!=0] = 1
prob_img = nib.Nifti1Image(prob_mask, nib.load(m).affine, nib.load(m).header)
nib.save(prob_img, m.split('_sub')[0]+'_'+emoPain+'_PROB25thesh_masked.nii.gz')

prob_mask[prob_mask<0.5] = 0
prob_mask[prob_mask!=0] = 1
prob_img = nib.Nifti1Image(prob_mask, nib.load(m).affine, nib.load(m).header)
nib.save(prob_img, m.split('_sub')[0]+'_'+emoPain+'_PROB50thesh_masked.nii.gz')

